# Global Parameters

In [21]:
startNet = ''
saveNetAs = 'start'
saveResultsAs = 'start.npy'
loadNet = 'start'
loadResult = 'start.npy'
saveTestAs = 'test.npy'
loadTest = 'test.npy'
valid_ratio = 0.3
batchSize = 512
numEpochs = 1
rangeEpochs = 10
learningRates = [0.0001, 0.001, 0.01]

import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

import os
oriPATH = 'C:/Users/Ryan/Desktop/machine-learning/part2/cnn'
task = '/final'
subFolder = '/round4'
if not os.path.exists(oriPATH + task):
    os.makedirs(oriPATH + task)
PATH = oriPATH + task + subFolder
if not os.path.exists(PATH):
    os.makedirs(PATH)

cuda:0


# Define CNN

In [22]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.lossFunction = nn.CrossEntropyLoss()
        self.act = nn.SELU()

        self.conv1 = nn.Conv2d(3, 48, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # self.norm1 = nn.BatchNorm2d(96)
        self.conv2 = nn.Conv2d(48, 96, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(96, 192, kernel_size=3, stride=1, padding=1)
        # self.norm2 = nn.BatchNorm2d(192)
        self.conv4 = nn.Conv2d(192, 192, kernel_size=3, stride=1, padding=1)
        self.drop1 = nn.AlphaDropout(p=0.05)
        # self.drop2 = nn.Dropout2d(0.05)
        self.fc1 = nn.Linear(192 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 10)
        
    def forward(self, x):
        x = self.pool(self.act(self.conv1(x)))
        x = self.act(self.conv2(x))
        # x = self.norm1(x)
        x = self.pool(self.act(self.conv3(x)))
        # x = self.norm2(x)
        x = self.act(self.conv4(x))
        x = x.view(-1, 192 * 8 * 8)
        x = self.act(self.fc1(x))
        x = self.act(self.fc2(x))
        x = self.act(self.fc3(x))
        x = self.fc4(x)
        return x


    def procBatch(self, batch):
        data, labels = batch
        pred = self(data)
        return self.lossFunction(pred, labels)
    
    def save(self, PATH, fileName, epoch):
        torch.save(self.state_dict(), os.path.join(PATH, "{}_epoch_{}.pth".format(fileName, epoch)))

    def load(self, PATH, fileName):
        self.load_state_dict(torch.load(os.path.join(PATH, fileName)))

# CIFAR-10 Dataset and Augmentations

In [23]:
import torchvision
import torchvision.transforms as transforms

dataTransform = transforms.Compose([
    transforms.RandomOrder([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.RandomAffine((-90, 90)),
        transforms.RandomResizedCrop(32)
    ]),
    transforms.ToTensor(),
    # Normalization numbers retrieved from https://github.com/kuangliu/pytorch-cifar/issues/19
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=dataTransform)
nb_train, nb_valid = int((1.0 - valid_ratio) * len(trainset)), int(valid_ratio * len(trainset))
train_dataset, valid_dataset = torch.utils.data.dataset.random_split(trainset, [nb_train, nb_valid])

trainLoader = torch.utils.data.DataLoader(train_dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=True)
validLoader = torch.utils.data.DataLoader(valid_dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=True)

testTransform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=testTransform)
testLoader = torch.utils.data.DataLoader(testset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=True)

Files already downloaded and verified
Files already downloaded and verified


# Train and Store

In [24]:
import numpy as np

# Training function
def train(numEpochs, model, optimizer, train, valid):
    bestEpoch = 0
    best_loss = np.float('inf')
    tally = []
    for epoch in range(numEpochs):
        runTrainLoss = []
        for _, data in enumerate(train, 0):
            optimizer.zero_grad()
            trainLoss = model.procBatch((data[0].to(device), data[1].to(device)))
            trainLoss.backward()
            optimizer.step()
            runTrainLoss.append(trainLoss)

        with torch.no_grad():
            runValidLoss = []
            for _, data in enumerate(valid, 0):
                runValidLoss.append(model.procBatch((data[0].to(device), data[1].to(device))))
        
        endTrain = torch.stack(runTrainLoss).mean()
        endValid = torch.stack(runValidLoss).mean()
        tally.append({'train': endTrain, 'valid': endValid})
        model.save(PATH, saveNetAs, epoch)

        valLoss = endValid.item()
        if endValid < best_loss:
            best_loss = endValid
            bestEpoch = epoch

        # stop early if it has been several epochs since last best
        if (epoch - bestEpoch) > rangeEpochs:
            break

    return tally

# Instantiate CNN and check params
model = Net()
if startNet != '':
    model.load(PATH, startNet)
model = model.to(device)
print(sum([p.numel() for p in model.parameters()]))

# Instantiate optimizer
modelOpt = torch.optim.Adam(model.parameters(), lr=0.001)

# Begin training
result = train(numEpochs, model, modelOpt, trainLoader, validLoader)
print(result[0])

# Save for future reference
import numpy as np
np.save(os.path.join(PATH, saveResultsAs), result)

6998442
{'train': tensor(2.1248, device='cuda:0', grad_fn=<MeanBackward0>), 'valid': tensor(1.9738, device='cuda:0')}
